## Reducer: (Put your name here)

## Reviewer: (Put your name here)

# jupyter notebook crash course

Click on a code cell (has grey background) then press Shift-Enter (at the same time) to run a code cell. That will add the controls (buttons, etc) you use to do the reduction one-by-one; then use them for reduction.

In [ ]:
from pathlib import Path

import reducer.gui
import reducer.astro_gui as astro_gui
from reducer.image_browser import ImageBrowser

from ccdproc import ImageFileCollection

from reducer import __version__
print(__version__)

# Enter name of directory that contains your data in the cell below, 

In [ ]:
data_dir = '/Users/mattcraig/Desktop/2018-07-23'


## Your calibrated images will go into a folder called "reduced"

In [ ]:
destination_dir = 'reduced'

path = Path(".") / destination_dir
path.mkdir(exist_ok=True)

# Type any comments about this dataset here

Double-click on the cell to start editing it.

# Load your data set

In [ ]:
images = ImageFileCollection(location=data_dir, keywords='*')

# Image Summary

## Includes browser and image/metadata viewer

This is not, strictly speaking, part of reduction, but is a handy way to take a quick look at your files.

In [ ]:
fits_browser = ImageBrowser(images, keys=['imagetyp', 'exposure'])
fits_browser.display()

# Make a master bias image

In [ ]:
def fix_trim(widget):
    the_field = widget.children[1:][0].children[2].children[1].children[0].children[4]
    the_field.value = 4096

## Reduce the bias images

In [ ]:
bias_reduction = astro_gui.Reduction(description='Reduce bias frames',
                                     toggle_type='button',
                                     allow_bias=False,
                                     allow_dark=False,
                                     allow_flat=False,
                                     input_image_collection=images,
                                     apply_to={'imagetyp': 'bias'},
                                     destination=destination_dir)
fix_trim(bias_reduction)
bias_reduction.display()

## Combine bias images to make master bias

In [ ]:
reduced_collection = ImageFileCollection(location=destination_dir, keywords='*')
bias_settings = astro_gui.Combiner(description="Master Bias Settings",
                                   toggle_type='button',
                                   file_name_base='master_bias',
                                   image_source=reduced_collection,
                                   apply_to={'imagetyp': 'bias'},
                                   destination=destination_dir)
bias_settings.display()

# Make a master dark

## Reduce dark images

In [ ]:
reduced_collection = ImageFileCollection(location=destination_dir, keywords='*')
dark_reduction = astro_gui.Reduction(description='Reduce dark frames',
                                     toggle_type='button',
                                     allow_bias=True,
                                     master_source=reduced_collection,
                                     allow_dark=False,
                                     allow_flat=False,
                                     input_image_collection=images,
                                     destination=destination_dir,
                                     apply_to={'imagetyp': 'dark'})
fix_trim(dark_reduction)
dark_reduction.display()

## Combine reduced darks to make master(s)

Note the ``Group by`` option in the controls that appear after you execute the cell below. `reducer` will make a master for each value of the FITS keyword listed in ``Group by``. By default this keyword is named ``exposure`` for darks, so if you have darks with exposure times of 10 sec, 15 sec and 120 sec you will get three master darks, one for each exposure time.

In [ ]:
reduced_collection = ImageFileCollection(location=destination_dir, keywords='*')

dark = astro_gui.Combiner(description="Make Master Dark",
                          toggle_type='button',
                          file_name_base='master_dark',
                          group_by='exposure',
                          image_source=reduced_collection,
                          apply_to={'imagetyp': 'dark'},
                          destination=destination_dir)
dark.display()

# Make master flats

## Reduce flat images

In [ ]:
reduced_collection = ImageFileCollection(location=destination_dir, keywords='*')
flat_reduction = astro_gui.Reduction(description='Reduce flat frames',
                                     toggle_type='button',
                                     allow_bias=True,
                                     master_source=reduced_collection,
                                     allow_dark=True,
                                     allow_flat=False,
                                     input_image_collection=images,
                                     destination=destination_dir,
                                     apply_to={'imagetyp': 'flat'})
fix_trim(flat_reduction)
flat_reduction.display()

## Build masters by combining

Again, note the presence of ``Group by``. If you typically use twilight flats you will almost certainly want to group by filter, not by filter and exposure.

In [ ]:
reduced_collection = ImageFileCollection(location=destination_dir, keywords='*')

flat = astro_gui.Combiner(description="Make Master Flat",
                          toggle_type='button',
                          file_name_base='master_flat',
                          group_by='filter',
                          image_source=reduced_collection,
                          apply_to={'imagetyp': 'flat'},
                          destination=destination_dir)
flat.display()

# Reduce the science images

There is some autmatic matching going on here:

+ If darks are subtracted a dark of the same edxposure time will be used, if available. If not, and dark scaling is enabled, the dark with the closest exposure time will be scaled to match the science image.
+ If the dark you want to scale appears not to be bias-subtracted an error will be raised.
+ Flats are matched by filter.

In [ ]:
reduced_collection = ImageFileCollection(location=destination_dir, keywords='*')
light_reduction = astro_gui.Reduction(description='Reduce light frames',
                                      toggle_type='button',
                                      allow_cosmic_ray=True,
                                      master_source=reduced_collection,
                                      input_image_collection=images,
                                      destination=destination_dir,
                                      apply_to={'imagetyp': 'light'})

fix_trim(light_reduction)
light_reduction.display()

## Wonder what the reduced images look like? Make another image browser...

In [ ]:
reduced_collection = ImageFileCollection(location=destination_dir, keywords='*')

In [ ]:
reduced_browser = ImageBrowser(reduced_collection, keys=['imagetyp', 'exposure'])
reduced_browser.display()